In [1]:
import pyvista 
from pyvista import examples
import os, os.path
import random
import numpy as np


### Function to generate random shadows of objects

In [3]:
def light_sampler(r):
        theta = np.random.uniform(0, np.pi/2)
        phi = np.random.uniform(0, 2*np.pi)
        return r*np.cos(phi)*np.sin(theta),r* np.sin(phi)*np.sin(theta), r*np.cos(theta) 
def sampler():
        rot_x = random.randint(0, 180)
        rot_y = random.randint(0, 180)
        rot_z = random.randint(0, 180)
        return rot_x, rot_y, rot_z 

def generate_samples(number_of_samples, name_of_object, mesh):
    pl = pyvista.Plotter()
    pyvista.global_theme.background = 'white'
     
    path = './Data/'+name_of_object+'/'
    os.makedirs(os.path.dirname('./Data/'), exist_ok=True)
    os.makedirs(os.path.dirname('./Data/'+name_of_object+'/'), exist_ok=True)
    for n in range(number_of_samples):
        rot_x, rot_y, rot_z = sampler()
        mesh.rotate_y(rot_x, inplace=True)
        mesh.rotate_x(rot_y, inplace=True)
        mesh.rotate_z(rot_z, inplace=True)
        pl = pyvista.Plotter()
        pl.add_mesh(mesh, show_edges=None, color='black') 
        pl.screenshot(path+"screenshot"+str(rot_x)+str(rot_y)+str(rot_z))
        pl.deep_clean()
    return

In [5]:
"""
#### The following is purley legacy code. NOT USED ANYWHERE

def generate_samples(number_of_samples, name_of_object, mesh,img_size=600):
    pl = pyvista.Plotter()
    pyvista.global_theme.background = 'white'
     
    path = './Data/'+name_of_object+'/'
    os.makedirs(os.path.dirname('./Data/'), exist_ok=True)
    os.makedirs(os.path.dirname('./Data/'+name_of_object+'/'), exist_ok=True)
    bounds = mesh.bounds
    rnge = (bounds[1] - bounds[0], bounds[3] - bounds[2], bounds[5] - bounds[4])

    for n in range(number_of_samples):
        rot_x, rot_y, rot_z = sampler()
        x, y, z = light_sampler(200*rnge[2])
        light_pos = np.array([x, y, z])
        light = pyvista.Light(
                position=(x, y, z ),#+70*bounds[5]
                focal_point=(0, 0, 0),
                color=[1.0, 1.0, 1.0, 1.0],  # Color temp. 5400 K
                intensity=10,
                )
        mesh.rotate_y(rot_x, inplace=True)
        mesh.rotate_x(rot_y, inplace=True)
        mesh.rotate_z(rot_z, inplace=True)
        pl = pyvista.Plotter(lighting=None, window_size=(img_size,img_size))
        pl.add_light(light)
        actor = pl.add_mesh(mesh, show_edges=None, color='white')
        c=-4*max(rnge)*light_pos/np.linalg.norm(light_pos)
        pl.add_mesh(pyvista.Plane(center=c, direction=(0,0,1),\
            i_size= 5*bounds[5], j_size= 5*bounds[5]))
        pl.enable_shadows()
        pl.set_position([c[0], c[1], 1*rnge[2]])
        camera = pyvista.Camera()
        pl.camera.focal_point = c
        #pl.show()
        pl.screenshot(path+"screenshot"+str(rot_x)+str(rot_y)+str(rot_z))
        pl.deep_clean()
    return
def generate_samples(number_of_samples, name_of_object, mesh):
    pl = pyvista.Plotter()
    pyvista.global_theme.background = 'white'
     
    path = './Data/'+name_of_object+'/'
    os.makedirs(os.path.dirname('./Data/'), exist_ok=True)
    os.makedirs(os.path.dirname('./Data/'+name_of_object+'/'), exist_ok=True)
    for n in range(number_of_samples):
        rot_x, rot_y, rot_z = sampler()
        mesh.rotate_y(rot_x, inplace=True)
        mesh.rotate_x(rot_y, inplace=True)
        mesh.rotate_z(rot_z, inplace=True)
        pl = pyvista.Plotter()
        pl.add_mesh(mesh, show_edges=None, color='black') 
        pl.screenshot(path+"screenshot"+str(rot_x)+str(rot_y)+str(rot_z))
        pl.deep_clean()
    return
"""

In [6]:
diamond = pyvista.read('./diamond.obj')
dodecahedron = pyvista.read('./dodecahedron.obj')
icosahedron = pyvista.read('./icosahedron.obj')
Tours = pyvista.read('./Tours.obj')
Cylinder = pyvista.Cylinder()
human = pyvista.read('./humanoid_quad.obj')

In [7]:
shapes = [pyvista.Cube(),pyvista.Tetrahedron(), examples.download_cow(), pyvista.Pyramid(),diamond,\
          dodecahedron, icosahedron, Tours, Cylinder, human]
names = ["Cube","Tetrahedron","cow","Pyramid","diamond", "dodecahedron","icosahedron", "Torus",\
        "Cylinder","human"]


### Generate Samples

In [8]:
#i=0
#generate_samples(200, names[i], shapes[i])

In [9]:
#i=1
#generate_samples(200, names[i], shapes[i])

In [10]:
#i=2
#generate_samples(200, names[i], shapes[i])

In [11]:
#i=3
#generate_samples(200, names[i], shapes[i])

In [12]:
#i=4
#generate_samples(100, names[i], shapes[i])

In [13]:
#i=5
#generate_samples(101, names[i], shapes[i])

In [14]:
#i=6
#generate_samples(100, names[i], shapes[i])

In [15]:
#i=7
#generate_samples(101, names[i], shapes[i])

In [16]:
#i=8
#generate_samples(100, names[i], shapes[i])

In [17]:
#i=9
#generate_samples(100, names[i], shapes[i])

### Convert dataset to numpy arrays

In [4]:
number_of_sammles = 400

In [1]:
from numba import jit
@jit(nopython=True)
def edge_detector(img, with_coordinates=False):
    res = img*0
    for a in range(1,len(img)-1):
        for b in range(1,len(img)-1):
            v = img[a,b]
            res[a,b] = bool(v^img[a+1,b-1]) or bool(v^img[a+1,b])\
            or bool(v^img[a+1,b+1]) or bool(v^img[a,b-1]) or bool(v^img[a,b])\
            or bool(v^img[a,b+1]) or bool(v^img[a-1,b-1]) or bool(v^img[a-1,b])\
            or bool(v^img[a-1,b+1])
            res[a,b] = not res[a,b]

    return res



In [2]:
#@jit(nopython=True)
def FD(img, length, norm='abs'):
    coordinates = []
    
    for a in range(len(img)):
        for b in range(len(img)):
            if bool(img[a,b]):
                coordinates.append(a +b*1j)
    #fd = np.fft.fft(coordinates)[0:length]
    c = []    
    for i in range(100):
        j = int(i *len(coordinates)/100)
        c.append(coordinates[j])
    fd = np.fft.fft(c)
    if norm=='abs':
        res = np.array([np.absolute(f)/np.absolute(fd[0])/np.absolute(fd[1]) for f in fd])
    if norm=='Cn':
        res = np.array([np.absolute(f/fd[0]) for f in fd])[1:]
        #im = np.array([np.imag(f/fd[1]/fd[0]) for f in fd])
        #res = np.concatenate([re, im])
    if norm=='Emil':
        res = normalize(np.absolute(fd[:,1:]))
    return res

In [3]:
labels = ["Cube","Tetrahedron","cow","Pyramid","diamond", "dodecahedron","icosahedron"\
          , "Torus", "Cylinder","human"]
img_size = 200
def get_data(data_dir, data_size,  data_type = 'edge'):
    FD_length = 50
    FD_norm='Cn'
    data = []
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for i in range(data_size):
            img=os.listdir(path)[i]
            try:
                img_arr = cv2.imread(os.path.join(path, img),cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                if data_type == 'edge':
                    edge_img = edge_detector(resized_arr)
                    data.append([edge_img, class_num])
                elif data_type == 'FD':
                    edge_img = edge_detector(resized_arr)
                    Fds = FD(edge_img, FD_length, FD_norm)
                    data.append([Fds, class_num])
                elif data_type == 'img':
                    data.append([resized_arr.astype(bool), class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [5]:
#data_set_edge = get_data(r'./Data',number_of_sammles, data_type = 'edge')

In [6]:
#data_set_img = get_data(r'./Data',number_of_sammles,data_type ='img')

In [7]:
data_set_FD = []
for img, label in data_set_edge:
    data_set_FD.append([FD(img, 100, norm='Cn')[1:], label])

In [8]:
#np.save(r'./Data/data_set_img',data_set_img)
#np.save(r'./Data/data_set_edge',data_set_edge)
#np.save(r'./Data/data_set_FD',data_set_FD)